In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 76.2 MB/s 
     |████████████████████████████████| 7.6 MB 79.1 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 95.9 MB/s 
     |████████████████████████████████| 212 kB 79.8 MB/s 
     |████████████████████████████████| 127 kB 80.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [5]:
from datasets import load_dataset

sts = load_dataset("mteb/sts13-sts")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/mteb___json/mteb--sts13-sts-a3d8d9f11c3fb4fb/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
"""sts['test'] -> 0.7: 0.2 : 0.1 (train, valid, test)"""

"sts['test'] -> 0.7: 0.2 : 0.1 (train, valid, test)"

In [17]:
nli = load_dataset("multi_nli")


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
nli['train'][0]['premise'], nli['train'][0]['hypothesis'], nli['train'][0]['label'] 

('Conceptually cream skimming has two basic dimensions - product and geography.',
 'Product and geography are what make cream skimming work. ',
 1)

In [20]:
from transformers import BertTokenizer, BertModel

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 1.3 MB 27.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3bd8409b61ec96472cd6e92fdc894ff33b3a300efbde8010cc41b7b6cb2d3a1c
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [23]:
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('all-MiniLM-L6-v2')


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [24]:
from torch.utils.data import Dataset, DataLoader
class SBertDataset(Dataset):
  def __init__(self, data):
    self.sentence1 = []
    self.sentence2 = []
    self.label = []
    for meta in data['test']:
      self.sentence1.append(meta['sentence1'])
      self.sentence2.append(meta['sentence2'])
      self.label.append(meta['score'])

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.sentence2[idx], self.label[idx]

In [25]:
class BertDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.sentence1 = []
    self.sentence2 = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data['test']:
      # BERT -> max 512 length
      self.sentence1.append(self.tokenizer(meta['sentence1'], max_length=128, padding='max_length', return_tensors='pt'))
      self.sentence2.append(self.tokenizer(meta['sentence2'], max_length=128, padding='max_length', return_tensors='pt'))
      self.label.append(meta['score'])

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.sentence2[idx], self.label[idx]

In [26]:
## BERT STS (cosine similarity)
epochs = 20
model = BertModel.from_pretrained('bert-base-uncased')
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
train_data = BertDataset(sts, tokenizer)
valid_data = BertDataset(sts, tokenizer)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
train_data.__getitem__(7)

({'input_ids': tensor([[  101,  1037,  9495, 12005, 26243,  2015,  1037,  3433,  1012,  1996,
           3433,  2089,  2593,  2022,  2070,  2785,  1997,  2895,  3214,  2000,
           4769,  1996,  9495,  2724,  2030,  2019,  5228,  8689,  1997,  1996,
           9495,  2724,  1012,   102,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,   

In [31]:
# 64, 1 , max_length

In [38]:
for epoch in range(1, epochs+1):
  cor = 0.0
  tot = 0.0
  for i, batch_data in enumerate(train_loader):
      optimizer.zero_grad()
      sents1, sents2, labels = batch_data
      # [batch_size, max_length, 768]
      sents1_embeddings = model(sents1['input_ids'].squeeze(1).to('cuda'), sents1['attention_mask'].squeeze(1), sents1['token_type_ids'].squeeze(1)).[:,0,:]
      sents2_embeddings = model(sents2['input_ids'].squeeze(1), sents2['attention_mask'].squeeze(1), sents2['token_type_ids'].squeeze(1))[:,0,:]
      # MSE loss
      loss = (((sents1_embeddings*sents2_embeddings).sum(dim=-1) - labels)**2).sum()
      loss.backward()
      optimizer.step()

  for i, batch_data in enumerate(valid_loader):
      sents1, sents2, labels = batch_data
      sents1_embeddings = model(sents1['input_ids'].squeeze(1), sents1['attention_mask'].squeeze(1), sents1['token_type_ids'].squeeze(1))[:,0,:]
      sents2_embeddings = model(sents2['input_ids'].squeeze(1), sents2['attention_mask'].squeeze(1), sents2['token_type_ids'].squeeze(1))[:,0,:]
      sim = (sents1_embeddings*sents2_embeddings).sum(dim=-1)
      cor += np.correlate(sim.cpu().numpy(), labels.cpu().numpy())
      tot += len(sents1)
  
  print (cor/tot)

SyntaxError: ignored

In [ ]:
sts

In [ ]:
## SBERT STS (cosine similarity)
epochs = 20
model = SentenceTransformer('all-MiniLM-L6-v2')
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
train_data = SBertDataset(sts)
valid_data = SBertDataset(sts)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

In [ ]:
train_data.__getitem__(7)

In [ ]:
model.encode(['a trigger elicits a response. the response may either be some kind of action meant to address the trigger event or an expressed judgment of the trigger event .', 'hihihihii']).shape

In [33]:
for epoch in range(1, epochs+1):
  cor = 0.0
  tot = 0.0
  for i, batch_data in enumerate(train_loader):
      optimizer.zero_grad()
      sents1, sents2, labels = batch_data
      sents1_embeddings = torch.tensor(model.encode(sents1), dtype=torch.float, requires_grad=True)
      sents2_embeddings = torch.tensor(model.encode(sents2), dtype=torch.float, requires_grad=True)
      # MSE loss
      loss = (((sents1_embeddings*sents2_embeddings).sum(dim=-1) - labels)**2).sum()
      loss.backward()
      optimizer.step()

  for i, batch_data in enumerate(valid_loader):
      sents1, sents2, labels = batch_data
      sents1_embeddings = torch.tensor(model.encode(sents1), dtype=torch.float)
      sents2_embeddings = torch.tensor(model.encode(sents2), dtype=torch.float)
      sim = (sents1_embeddings*sents2_embeddings).sum(dim=-1)
      cor += np.correlate(sim.cpu().numpy(), labels.cpu().numpy())
      tot += len(sents1)
  
print (cor/tot)

AttributeError: ignored

In [34]:
from torch.utils.data import Dataset, DataLoader


In [35]:
class SentenceClassificationModelNLI(nn.Module):
  def __init__(self):
    super(SentenceClassificationModelNLI, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # CLS token x class nums
    self.classification_layer = nn.Linear(768, 3)
  
  def forward(self, input_ids, attention_mask, token_type_ids):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = bert_output[:,0,:]
    pred = self.classification_layer(cls_token)
    return pred

In [ ]:
class BertDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.sentence = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data:
      self.sentence.append(self.tokenizer(meta['premise'], meta['hypothesis'],  max_length=256, padding='max_length', truncation=True, return_tensors='pt'))
      self.label.append(meta['label'])

  def __len__(self):
    return len(self.sentence)

  def __getitem__(self, idx):
    return self.sentence[idx], self.label[idx]

In [ ]:
nli

In [ ]:
## NLI (multi-class classification)
epochs = 20
model = SentenceClassificationModelNLI()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
train_data = BertDataset(nli['validation_matched'], tokenizer)
valid_data = BertDataset(nli['validation_matched'], tokenizer)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)
loss_func = nn.CrossEntropyLoss()

In [ ]:
train_data.__getitem__(7)

In [ ]:
for epoch in range(1, epochs+1):
  acc = 0.0
  tot = 0.0
  for i, batch_data in enumerate(train_loader):
      optimizer.zero_grad()
      sents, labels = batch_data
      pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
      loss = loss_func(pred, labels)
      loss.backward()
      optimizer.step()

  for i, batch_data in enumerate(valid_loader):
      sents, labels = batch_data
      pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
      acc += (torch.argmax(pred, dim=-1), labels).sum()
      tot += 1
  
  print (acc/tot)

In [ ]:
sem_eval = load_dataset("sem_eval_2018_task_1",'subtask5.english')

In [ ]:
sem_eval

In [29]:
sem_eval['train'][0]['Tweet']

"“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"

In [30]:
sem_eval['train'][0]['anger']

False

In [11]:
sem_eval['train'][0]['anticipation']

True

In [12]:
class BertDataset(Dataset):
  def __init__(self, data, tokenizer):
    self.sentence = []
    self.label = []
    self.tokenizer = tokenizer
    for meta in data:
      self.sentence.append(self.tokenizer(meta['Tweet'], max_length=256, padding='max_length', truncation=True, return_tensors='pt'))
      temp = []
      for key in meta.keys():
        if key in ['ID', 'Tweet']:
          continue
        if meta[key] == True:
          temp.append(1)
        else:
          temp.append(0)
      self.label.append(temp)

  def __len__(self):
    return len(self.sentence)

  def __getitem__(self, idx):
    return self.sentence[idx], self.label[idx]

NameError: ignored

In [13]:
class SentenceClassificationModelMultiClass(nn.Module):
  def __init__(self):
    super(SentenceClassificationModelMultiClass, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # num of class (11) -> 0, 1 BCE loss
    self.classification_layer = nn.Linear(768, 11)
  
  def forward(self, input_ids, attention_mask, token_type_ids):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = bert_output[:,0,:]
    pred = self.classification_layer(cls_token)
    pred = F.sigmoid(pred)
    return pred

In [14]:
# multi label classification
epochs = 20
model = SentenceClassificationModelMultiClass()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
train_data = BertDataset(sem_eval['train'], tokenizer)
valid_data = BertDataset(sem_eval['validation'], tokenizer)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)
loss_func = nn.BCELoss()

NameError: ignored

In [15]:
train_data.__getitem__(7)

NameError: ignored

In [16]:
for epoch in range(1, epochs+1):
  valid_loss = 0.0
  tot = 0.0
  for i, batch_data in enumerate(train_loader):
    optimizer.zero_grad()
    sents, labels = batch_data
    pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
    loss = loss_func(pred, labels)
    loss.backward()
    optimizer.step()

  for i, batch_data in enumerate(valid_loader):
    sents, labels = batch_data
    pred = model(sents['input_ids'].squeeze(1), sents['attention_mask'].squeeze(1), sents['token_type_ids'].squeeze(1))
    loss = loss_func(pred, labels)   
    valid_loss += loss.item()
    tot += len(sents)

valid_loss/tot

NameError: ignored

In [ ]:
# 1. Metric learning (MSE loss) -> 다음강의 검색 모델 사용.!! 2. Multi-class classification (cross entropy loss), 3. Multi-label classification (Binary cross entropy loss)